### Reducers

- **Reducers** are key to understanding how updates from nodes are applied to the **State**.  
- Each **key in the State** has its own independent **reducer function**.  
- If no reducer function is explicitly specified, the default behavior is that all updates to that key **override** its previous value.  
- There are a few different types of reducers:
  1. **Pre-defined reducer functions**
  2. **Custom reducer functions**


<p align="center">
  <img src="langgraph_reducer_final.gif" width="800">
</p>

#### Default

In [ ]:
from typing_extensions import TypedDict

In [ ]:
class TypedState(TypedDict):
    name : str
    topics : list[str]

In [ ]:
def node_a(state):
    print("Node A received state:", state)
    return {"name" : state['name'] + 'Learner', "topics" : ["langgraph"]}

def node_b(state):
    print("Node B received state:", state)
    return {"topics" : ['langchain']}

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image # type: ignore

In [ ]:
builder = StateGraph(TypedState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
builder.add_edge("node_a", "node_b")
builder.add_edge("node_b", END)

graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"name": "AI", "topics": []})

Branching

In [ ]:
builder = StateGraph(TypedState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
builder.add_edge(START, "node_b")
builder.add_edge("node_a", END)
builder.add_edge("node_b", END)

graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"name": "AI", "topics": []})

#### Pre-defined Reducer functions

In [ ]:
from typing import Annotated
from operator import add

In [ ]:
class ReducerState(TypedDict):
    name : str
    topics : Annotated[list[str], add]

In [ ]:
builder = StateGraph(ReducerState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
builder.add_edge(START, "node_b")
builder.add_edge("node_a", END)
builder.add_edge("node_b", END)

graph = builder.compile()

graph.invoke({"name": "AI", "topics": []})

In [ ]:
graph.invoke({"name": "AI", "topics": None}) # type: ignore

#### Custom reducer functions

In [ ]:
def custom_reducer(left, right):
    if not left:
        return right
    if not right:
        return left
    return left + right

In [ ]:
class ReducerState(TypedDict):
    name : str
    topics : Annotated[list[str], custom_reducer]

In [ ]:
builder = StateGraph(ReducerState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
builder.add_edge(START, "node_b")
builder.add_edge("node_a", END)
builder.add_edge("node_b", END)

graph = builder.compile()

graph.invoke({"name": "AI", "topics": None}) # type: ignore

#### Chat Messages


- Most modern **LLM providers** support a **chat model interface** that accepts a list of messages as input.  
- In **LangChain**, the `ChatModel` takes a list of **Message** objects such as:
  - `HumanMessage` → user input  
  - `AIMessage` → model response  
- These message objects enable structured, multi-turn interactions between users and the model.  

---

##### Using Messages in Your Graph
- Prior conversation history can be stored as a **list of messages** in your **graph state**.  
- To do this, add a **key (channel)** to the state that holds a list of `Message` objects.  
- Annotate this key with a **reducer function** to control how messages are updated.  
- If no reducer is specified, **new updates overwrite** the previous list of messages.  
- To **append** messages (i.e., keep the conversation history), use: reducer=add_messages from langgraph


In [ ]:
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

In [ ]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
inp_msg = [HumanMessage(content="Hi")]
add_messages(inp_msg, AIMessage(content="Hello! How can I assist you today?")) # type: ignore

Overwriting and removal

In [ ]:
inp_msg = [HumanMessage(content="Hi", name="user", id="1"),
           AIMessage(content="Hello", name="assistant", id="2")]
add_messages(inp_msg, AIMessage(content="Hello! How can I assist you today?", name="assistant", id="2")) # type: ignore

In [ ]:
from langchain_core.messages import RemoveMessage

In [ ]:
inp_msg = [HumanMessage(content="Hi", name="user", id="1"),
           AIMessage(content="Hello", name="assistant", id="2"),
           HumanMessage(content="Who will win fifa world cup 2026?", name="user", id="3"),
           AIMessage(content="I don't know yet as this is a future event.", name="assistant", id="4")]

In [ ]:
deleted_msgs = [RemoveMessage(m.id) for m in inp_msg if m.id in ["1", "2"]]
deleted_msgs

In [ ]:
add_messages(inp_msg, deleted_msgs) # type: ignore

#### MessageState

In [ ]:
from langgraph.graph import MessagesState

In [ ]:
def assistant(state):
    return {"messages" : [AIMessage(content="Hello! How can I assist you today?")]}

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)

builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"messages": [HumanMessage(content="Hi!")]})

#### Real-life examples utilizing reducer functions

##### 1. Metasearch Result Merger
- Combines results from multiple search engines or data sources.  
- The reducer merges, deduplicates, and ranks search results by relevance or confidence score.

---

##### 2. Map-Reduce Summarization
- Each node summarizes a different chunk or section of a document.  
- The reducer combines these partial summaries into a single cohesive summary.

---

##### 3. Ensembling Model Outputs
- Run Multiple LLM/prompts in parallel
- Several LLM nodes produce alternative answers to the same query.  
- The reducer compares outputs, scores them, and keeps the best or consensus response.

---

##### 4. Multi-Vendor Price Aggregation
- Used when multiple vendor APIs return prices for the same product.  
- The reducer merges or averages the results to produce a unified price list or select the lowest price.

---